In [25]:
import pandas as pd

dat = pd.read_csv("../data/msr-paraphrase/msr_combined.csv")

# train = "../data/msr-paraphrase/msr-para-train.tsv"
# val = "../data/msr-paraphrase/msr-para-val.tsv"
# test = "../data/msr-paraphrase/msr-para-test.tsv"

# def read_msr(filepath: str, source: str) -> pd.DataFrame:
#     rows = []
#     with open(filepath, "r") as f:
#         lines = f.readlines()
        
#     for line in lines[1:]:
#         line = line.split(sep="\t")
#         rows.append({"label": line[0],
#                      "text1_id": line[1],
#                      "text2_id": line[2],
#                      "text1": line[3],
#                      "text2": line[4],
#                      "source": source})
        
#     return pd.DataFrame(rows)
        
# train = read_msr(train, "train")
# val = read_msr(val, "validation")
# test = read_msr(test, "test")

# dat = pd.concat([train,val,test], axis=0).reset_index()
# dat.to_csv("../data/msr-paraphrase/msr_combined.csv", index_label=False, index=False)

In [34]:
train = dat.query("source=='train'")
val = dat.query("source=='validation'")
test = dat.query("source=='test'")

## Simple Basline

Goal: Predict if 2 sentences are paraphrase of each other.

### Approach

Word Rep:
    - TFIDF, GLOVE, InferSENT

Features:
    - cosine sim
    - spacy sim

Models:
    - Simple logit
    - random forest 
    - SVM

In [68]:
# Model setup

from sklearn.feature_extraction.text import TfidfVectorizer

# Stack all sents in train and valid to train vectorizire
# all_sents = train["text1"].tolist()
# all_sents.extend(train["text2"].tolist())
# all_sents.extend(val["text1"].tolist())
# all_sents.extend(val["text2"].tolist())

# with open("../data/msr-paraphrase/all_sents.txt", "w") as f:
#     for sent in all_sents:
#         f.write(sent + "\n")
        
with open("../data/msr-paraphrase/all_sents.txt", "r") as f:
    all_sents = f.readlines()

In [105]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf = tfidf.fit(all_sents)


In [83]:
train["combined_input"] = train.apply(lambda x: x["text1"] + " [SEP] " + x["text2"], axis=1)

/home/doc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [99]:
vec = tfidif.transform(train["combined_input"])
y = train["label"]

from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5, max_iter = 500)
clf.fit(vec, y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=500, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [102]:
val["combined_input"] = val.apply(lambda x: x["text1"] + " [SEP] " + x["text2"], axis=1)
val_vec = tfidf.transform(val["combined_input"])

clf.score(val_vec, val["label"])

/home/doc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0.706

In [123]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf = tfidf.fit(all_sents)


def encode_inputs(df: pd.DataFrame, tfidf: TfidfVectorizer) -> pd.Series:
    df["combined_input"] = df.apply(lambda x: x["text1"] + " [SEP] " + x["text2"], axis=1)
    return tfidf.transform(df["combined_input"])

X_train = encode_inputs(train, tfidf)
X_val = encode_inputs(val, tfidf)
X_test = encode_inputs(test,tfidf)

y_train = train["label"]
y_val = val["label"]
y_test = test["label"]

/home/doc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [126]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

models = [("logistic", LogisticRegressionCV(cv=10, max_iter = 500)),
          ("random forest", RandomForestClassifier()),
          ("svm", LinearSVC())]
trained_models = {}
for model in models:
    name = model[0]
    clf  = model[1]
    print("Evaluating ", name)
    
    # 1. Fit model
    clf.fit(X_train, y_train)
    trained_models["name"]=clf
    
    # 2. predict against test
    preds = clf.predict(X_test)
    
    # 3. Classifcation report
    print(classification_report(y_test, preds))
    
    print("------------------------------------ \n")


Evaluating  logistic
              precision    recall  f1-score   support

           0       0.57      0.24      0.33       578
           1       0.70      0.91      0.79      1147

    accuracy                           0.68      1725
   macro avg       0.64      0.57      0.56      1725
weighted avg       0.66      0.68      0.64      1725

------------------------------------ 

Evaluating  random forest


/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.52      0.27      0.36       578
           1       0.70      0.87      0.78      1147

    accuracy                           0.67      1725
   macro avg       0.61      0.57      0.57      1725
weighted avg       0.64      0.67      0.64      1725

------------------------------------ 

Evaluating  svm
              precision    recall  f1-score   support

           0       0.50      0.35      0.41       578
           1       0.72      0.83      0.77      1147

    accuracy                           0.67      1725
   macro avg       0.61      0.59      0.59      1725
weighted avg       0.64      0.67      0.65      1725

------------------------------------ 



In [141]:
# test out baselines against kgap
kgap = pd.read_csv("../data/kgap/kgap_semantic_sim.csv")
all_ke = pd.read_csv("../data/kgap/ke_lookup_table.csv")

In [181]:
from sklearn.feature_extraction.text import TfidfVectorizer

content = all_ke["content"].dropna()

tfidf = TfidfVectorizer(stop_words="english")
tfidf = tfidf.fit(content)

def encode_inputs(df: pd.DataFrame, tfidf: TfidfVectorizer) -> pd.Series:
    df["combined_input"] = df.apply(lambda x: x["text1"] + " [SEP] " + x["text2"], axis=1)
    return tfidf.transform(df["combined_input"])


kgap["text1"] = kgap["text1"].apply(lambda x: str(x).strip())
kgap["text2"] = kgap["text2"].apply(lambda x: str(x).strip())

dat = kgap[["text1", "text2", "similar", "ora"]].dropna()

In [182]:
X = encode_inputs(kgap, tfidf)
y = kgap["similar"]

In [183]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = .2, 
                                                    stratify=y)


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

models = [("logistic", LogisticRegressionCV(cv=10, max_iter = 1000)),
          ("random forest", RandomForestClassifier()),
          ("svm", LinearSVC())]

trained_models = {}

for model in models:
    name = model[0]
    clf  = model[1]
    print("Evaluating ", name)
    
    # 1. Fit model
    clf.fit(X_train, y_train)
    trained_models["name"]=clf
    
    # 2. predict against test
    preds = clf.predict(X_test)
    
    # 3. Classifcation report
    print(classification_report(y_test, preds))
    
    print("------------------------------------ \n")


Evaluating  logistic


/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of

              precision    recall  f1-score   support

           0       0.83      0.77      0.80      1793
           1       0.79      0.84      0.81      1793

    accuracy                           0.81      3586
   macro avg       0.81      0.81      0.81      3586
weighted avg       0.81      0.81      0.81      3586

------------------------------------ 

Evaluating  random forest


/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.78      0.81      0.80      1793
           1       0.80      0.78      0.79      1793

    accuracy                           0.79      3586
   macro avg       0.79      0.79      0.79      3586
weighted avg       0.79      0.79      0.79      3586

------------------------------------ 

Evaluating  svm
              precision    recall  f1-score   support

           0       0.79      0.79      0.79      1793
           1       0.79      0.79      0.79      1793

    accuracy                           0.79      3586
   macro avg       0.79      0.79      0.79      3586
weighted avg       0.79      0.79      0.79      3586

------------------------------------ 



In [184]:
from sklearn.feature_extraction.text import TfidfVectorizer

content = all_ke["content"].dropna()

tfidf = TfidfVectorizer(stop_words="english")
tfidf = tfidf.fit(content)

def encode_inputs(df: pd.DataFrame, tfidf: TfidfVectorizer) -> pd.Series:
    df["combined_input"] = df.apply(lambda x: x["text1"] + " [SEP] " + x["text2"], axis=1)
    return tfidf.transform(df["combined_input"])


kgap["text1"] = kgap["text1"].apply(lambda x: str(x).strip())
kgap["text2"] = kgap["text2"].apply(lambda x: str(x).strip())

dat = kgap[["text1", "text2", "similar", "ora"]].dropna()

,text1,text2,similar
0,How to I set an out of the office reply on my ...,How to I set an out of the office reply on my ...,1
1,The location is wrong for my Square transactions.,Tab name is printing small,0
2,How to I set an out of the office reply on my ...,How do you set up an out of office reply,1
3,in the analytics report under the payment summ...,why are the delivery charges not automatically...,0
4,Payment instructions,How do I edit the mailing address payment inst...,1
5,what to do if azpen force freezes,Can I take an address change request for a pol...,0
6,Payment instructions,How do I edit the payment instructions on the ...,1
7,customer needs a copy of the hardware invoice,how to set up a job code to not take place in ...,0
8,Checking Rewards Balance,how to view loyalty balance,1
9,what do we do if there is a case by case situa...,I lost my link to bid. Where can I bid?,0


In [231]:
# kgap["text1"] = kgap["text1"].apply(lambda x: str(x).strip())
# kgap["text2"] = kgap["text2"].apply(lambda x: str(x).strip())
# kgap.to_csv("../data/kgap/kgap_semantic_sim.csv")

# kgap = pd.read_csv("../data/kgap/kgap_semantic_sim.csv",  dtype={'organization_id': object})
# kgap["organization_id"] = kgap["organization_id"].fillna("00000")

# train, test = train_test_split(kgap, test_size=.2, stratify=kgap[["similar", "organization_id"]])

# train, val = train_test_split(train, test_size=.2, stratify=train[["similar", "organization_id"]])

In [234]:
# test.to_csv("../data/kgap/kgap_test.csv", index=False)
# train.to_csv("../data/kgap/kgap_train.csv", index=False)
# val.to_csv("../data/kgap/kgap_validation.csv", index=False)

In [237]:
kgap_train = pd.read_csv("../data/kgap/kgap_train.csv")
kgap_test  = pd.read_csv("../data/kgap/kgap_test.csv")
kgap_val   = pd.read_csv("../data/kgap/kgap_validation.csv")

In [247]:
# import pickle
# pickle.dump(tfidf, open("kgap_tfidf.pkl", "wb"))

X_train = tfidf.transform(kgap_train["combined_input"])
y_train = kgap_train["similar"]

X_test = tfidf.transform(kgap_test["combined_input"])
y_test = kgap_test["similar"]

In [253]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

models = [("logistic", LogisticRegressionCV(cv= 5, max_iter = 1000)),
          ("random forest", RandomForestClassifier())]

trained_models = {}

for model in models:
    name = model[0]
    clf  = model[1]
    print("Evaluating ", name)
    
    # 1. Fit model
    clf.fit(X_train, y_train)
    trained_models[name]=clf
    
    # 2. predict against test
    preds = clf.predict(X_test)
    
    # 3. Classifcation report
    print(classification_report(y_test, preds))
    
    print("------------------------------------ \n")


Evaluating  logistic


/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of

              precision    recall  f1-score   support

           0       0.81      0.75      0.78      1793
           1       0.76      0.82      0.79      1793

    accuracy                           0.78      3586
   macro avg       0.79      0.78      0.78      3586
weighted avg       0.79      0.78      0.78      3586

------------------------------------ 

Evaluating  random forest


/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.78      0.79      0.78      1793
           1       0.78      0.78      0.78      1793

    accuracy                           0.78      3586
   macro avg       0.78      0.78      0.78      3586
weighted avg       0.78      0.78      0.78      3586

------------------------------------ 



In [263]:
orgs = kgap_test["organization_id"].unique()

for model, clf in trained_models.items():
    for org in orgs:
        print(org)
        dat = kgap_test.query(f"organization_id=={org}")

        X = tfidf.transform(dat["combined_input"])
        y = dat["similar"]
        
        preds = clf.predict(X)
        print(classification_report(y,preds))
        print("================================")
    
    print("----------------------")

71988
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.82      0.90       351

    accuracy                           0.82       351
   macro avg       0.50      0.41      0.45       351
weighted avg       1.00      0.82      0.90       351

0
              precision    recall  f1-score   support

           0       1.00      0.75      0.86      1793
           1       0.00      0.00      0.00         0

    accuracy                           0.75      1793
   macro avg       0.50      0.37      0.43      1793
weighted avg       1.00      0.75      0.86      1793

72075
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.82      0.90       692

    accuracy                           0.82       692
   macro avg       0.50      0.41      0.45       692
weighted avg       1.00      0.82      0.90       692

81444

/home/doc/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      0.79      0.88      1793
           1       0.00      0.00      0.00         0

    accuracy                           0.79      1793
   macro avg       0.50      0.39      0.44      1793
weighted avg       1.00      0.79      0.88      1793

72075
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.78      0.87       692

    accuracy                           0.78       692
   macro avg       0.50      0.39      0.44       692
weighted avg       1.00      0.78      0.87       692

81444
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.73      0.84       126

    accuracy                           0.73       126
   macro avg       0.50      0.37      0.42       126
weighted avg       1.00      0.73      0.84       126

72191
 